# Benchmarking of different sampling algorithms

This notebooks assesses the performance of different sampling algorithms available in Python. The task is to sample `k` items from a csv file stored on the disk. As a baseline, we compare to the speed of reading/parsing a csv file from the disk (without sampling from it).

We record the execution time as well as the bandwith in MB/sec.

We use `csv`, `pandas` and `dask` to read and parse the csv files. Using `dask` is advisable since it implements the parallelisation concepts similar to those in `elasticsearch` and hence the sampling algorithms implemented on top of `dask` will be easier to port later.

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

from incremental_learning.storage import download_dataset
from incremental_learning.misc import DataStream, reservoir_sample_with_jumps
from incremental_learning.config import datasets_dir

import pandas as pd
import dask
import dask.dataframe as dd

import time
import csv


In [2]:
# dataset_name = 'autompg'
dataset_name = 'ember_malware_bytes'
dataset = datasets_dir / "{}.csv".format(dataset_name)
size_mb = dataset.stat().st_size/1024/1024

In [3]:
def bandwidth(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        elapsed_time = time.time() - start_time
        bandwidth = size_mb/elapsed_time
        print("Elapsed time: {} sec\nBandwidth: {} MB/sec" .format(elapsed_time, bandwidth))
        return res
    return wrapper

In [4]:
@bandwidth
def dictreader():
    """Read and parse the csv file using csv library."""
    with open(dataset, newline='') as csvfile:
        reader = csv.DictReader(csvfile, dialect='unix')
        for row in reader:
            continue
            
@bandwidth
def csvreader():
    """Read only the csv file using the csv librar."""
    with open(dataset, newline='') as csvfile:
        reader = csv.reader(csvfile, dialect='unix')
        for row in reader:
            continue
            
@bandwidth
def pdreader():
    """Read the csv file using pandas library."""
    df = pd.read_csv(dataset)

@bandwidth
def reservoir(k):
    """Read from file and sample using reservoir sample la"""
    D = reservoir_sample_with_jumps(dataset, k)
    return D
    
@bandwidth
def pdsampler(k):
    """Read the csv file with panda and sample randomly."""
    df = pd.read_csv(dataset)
    return df.sample(frac=k)

@bandwidth
def ddsampler(k):
    """Read the csv file with dusk and sample randomly."""
    df = dd.read_csv(dataset)
    return df.sample(frac=k).compute()

@bandwidth
def ddsortchoose(k):
    """Read the csv file with dusk, sort it and choose k maximum elements."""
    df = dd.read_csv(dataset)
    df = df.set_index('byte_histogram_0')
    return df.tail(k)

@bandwidth
def ddnlargest(k):
    """Read the csv file with dusk and select k largest elements."""
    df = dd.read_csv(dataset)
    return df.nlargest(n=k, columns=['byte_histogram_0']).compute()


In [5]:
dictreader()

Elapsed time: 40.47589683532715 sec
Bandwidth: 76.29892688511258 MB/sec


In [6]:
csvreader()

Elapsed time: 25.200711488723755 sec
Bandwidth: 122.54683740297784 MB/sec


In [7]:
pdreader()

Elapsed time: 24.42076063156128 sec
Bandwidth: 126.46074132747214 MB/sec


In [8]:
_ = reservoir(10000)

Elapsed time: 46.38114285469055 sec
Bandwidth: 66.58454930537934 MB/sec


In [9]:
_ = pdsampler(0.2)

Elapsed time: 23.54236626625061 sec
Bandwidth: 131.17914564413186 MB/sec


In [10]:
_ = ddsampler(0.2)

Elapsed time: 10.396746158599854 sec
Bandwidth: 297.0417326860934 MB/sec


In [11]:
_ = ddsortchoose(2000)

Elapsed time: 24.716933250427246 sec
Bandwidth: 124.94541543476488 MB/sec


In [12]:
_ = ddnlargest(2000)

Elapsed time: 11.367468357086182 sec
Bandwidth: 271.67592609332763 MB/sec


Using the `dask` library brings us close to the bandwidth of simply reading a file onto and SSD drive! `ddsampler` processes the `ember` dataset in about 10 seconds, however, it is unable to do any weighted sampling. With `ddnlargest` we process `ember` in 11 seconds and it is only slightly slower than `ddsampler`, while it is able to use a column of weights to select samples with the highest indicator.

As a downside, `ddnlargest` can sample clustered points. As a remedy, we could run `diversipy` psa sampler on top of the reduced sample (e.g. having 5 `k` samples)  to increase the sample diversity.